**2024.11.27**

#데이터 전처리 2
    1. 관광지명
    2. 소재지(도로명 주소)
    3. 공공편익시설정보, 휴양및문화시설정보
    4. 관광지소개


###prompt (모델 학습 순서)
    1. 지역별 추천 여행지
    2. 테마별 추천 여행지 (유적지, 식물원, 액티비티)
    3. 관광지 소개
    4. 관광지 주소

    **(질문-답변 예시)**
    1. {충청남도 홍성군}에 갈만한 여행지 추천해줘 - 주소 앞에 두 개만 잘라서
      - {관광지명}, {관광지소개}
    2. {공공편익시설정보, 휴양및문화시설정보} 가 있는 여행지 추천해줘
      - {관광지명}, {관광지소개}
    3. {관광지명}에 대해 설명해줘
      - {관광지소개}  
    4. {관광지명}에 대해 주소가 뭐야
	    - {소재지도로명주소}  


###NULL 데이터 처리
    1. 필수 필드 아닌 경우에 대한 처리
        e.g. 휴양및문화시설정보, 공공편익시설정보가 모두 없는 경우

###1. 말뭉치 데이터 전처리

In [ ]:
import pandas as pd

# Google Drive에 저장한 json 파일 불러오기
import json

from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/corpus.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# 필요한 정보만 추출
# 1. 4개 정보 중 하나라도 없으면 패스
# 2. 카테고리는 두 글자 이상인 경우만 유의미한 데이터로 취급
processed_data = [
    {
        "name": item["관광지명"],
        "region": item.get("소재지도로명주소"),
        "category": ", ".join(filter(None, [item.get("휴양및문화시설정보"), item.get("공공편익시설정보")])),
        "description": item.get("관광지소개")
    }
    for item in data.get("records", [])
    if item.get("관광지명") and item.get("소재지도로명주소") and item.get("관광지소개") and
       len(item.get("휴양및문화시설정보")+item.get("공공편익시설정보")) >= 3
]

# 데이터 프레임으로 변환 (시각적 확인 및 처리 용이)
# df = pd.DataFrame(processed_data)

# 전처리 완료된 데이터 저장
file_path = "/content/sample_data/processed_travel_data.json"

with open(file_path, "w", encoding='utf-8') as file:
    # ensure_ascii=False를 추가하여 한글을 그대로 저장
    json.dump(processed_data, file, ensure_ascii=False, indent=4)
    print("데이터 전처리 JSON 파일이 성공적으로 저장되었습니다.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
데이터 전처리 JSON 파일이 성공적으로 저장되었습니다.


###1-2. 학습데이터 준비

In [ ]:
import pandas as pd

# Google Drive에 저장한 json 파일 불러오기
import json

from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/processed_travel_data.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# Prompt와 Response 생성
prompt_response_pairs = [
    # {
    #     "prompt": f"{' '.join(row['region'].split()[:2])}에서 갈만한 여행지 추천해줘.",
    #     "response": f"{row['name']} - {row['description']}"
    # }
    # {
    #     "prompt": f"{row['category']} 있는 여행지 추천해줘.",
    #     "response": f"{row['name']} - {row['description']}"
    # }
    # {
    #     "prompt": f"{row['name']} 대해 설명해줘.",
    #     "response": f"{row['description']}"
    # }
    {
        "prompt": f"{row['name']} 의 주소가 뭐야.",
        "response": f"{row['region']}"
    }
    for row in data
    # if item.get("관광지명") and item.get("소재지도로명주소") and item.get("관광지소개") and
    #    len(item.get("휴양및문화시설정보")+item.get("공공편익시설정보")) >= 3


    # **(질문-답변 예시)**
    # 1. {충청남도 홍성군}에 갈만한 여행지 추천해줘 - 주소 앞에 두 개만 잘라서
    #   - {관광지명} - {관광지소개}
    # 2. {공공편익시설정보, 휴양및문화시설정보} 가 있는 여행지 추천해줘
    #   - {관광지명}, {관광지소개}
    # 3. {관광지명}에 대해 설명해줘
    #   - {관광지소개}
    # 4. {관광지명}에 대해 주소가 뭐야
	  #   - {소재지도로명주소}
]

# 학습 데이터 저장
file_path = "/content/sample_data/travel_chatbot_data(4).json"

with open(file_path, "w", encoding='utf-8') as file:
    json.dump(prompt_response_pairs, file, ensure_ascii=False, indent=4)
    print("학습 데이터 JSON 파일이 성공적으로 저장되었습니다.")

# 샘플 데이터 확인
print(prompt_response_pairs[:2])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
학습 데이터 JSON 파일이 성공적으로 저장되었습니다.
[{'prompt': '김포국제조각공원 의 주소가 뭐야.', 'response': '경기도 김포시 월곶면 용강로13번길 38'}, {'prompt': '아트빌리지(한옥마을) 의 주소가 뭐야.', 'response': '경기도 김포시 모담공원로 170'}]


###2. 모델 학습 및 추론

In [ ]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# KoGPT 모델 및 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>')
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # padding token 추가
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))  # 모델 임베딩 크기 조정

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prompt 예제
prompt = "서울에서 갈만한 여행지 1곳 추천해줘."

# 입력 토큰 생성
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

# KoGPT로 텍스트 생성
output = model.generate(
    input_ids,
    max_new_tokens=200,  # 새로 생성할 토큰 길이
    num_return_sequences=1,
    do_sample=True,     # 랜덤 샘플링 활성화
    top_k=50,           # 상위 K개의 토큰만 고려
    top_p=0.95,          # 상위 P 확률의 토큰만 고려
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id
)

# 결과 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print("Generated Text:", generated_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Generated Text: 서울에서 갈만한 여행지 1곳 추천해줘. 우리 여행사에도 소개해주고 있으니까 더 많이 추천해드려야겠어."
"아마도 여러 번 방문했던 곳인데 꼭 가서 여행하는 것도 좋을 것 같습니다. 괜찮겠지?"
이때의 고민을 물으니 내 눈엔 한 가지도 없었다.
일단 그곳엔 관광지 몇 곳이 있는지 확인해봤더니 모두 그곳에 있었고, 내가 직접 가볼 만한 곳이 눈에 띄었다.
이곳에서 나의 안내를 받아 떠나는 여행이라면 그 곳을 찾아다니는 것이 좋겠고, 내가 다녀온 곳을 돌아볼 수도 있는 여행이라면 그 여행이라면 꼭 가보아야 하는 여행이라면 그곳은 그 여행의 종착지라고 할 수 있겠고, 그러니까 나의 여행이라면 적어도 그곳이 나의 목적지라는 것은 자명할 수 있는 것이다.
아마 내 여행은 이렇게 될 것이다.
내 여행이 목적지를 벗어나고 나면 내 여행은 끝이 나는 것이다.
그곳에 가면 나의 여행은 끝이 난다.
우리 모두는 모두의 여행지
내 여행이 목적지를 벗어나서 목적지로 돌아갈 때 그 여행을


###3. Fine-Tuning

In [ ]:
!pip -q install datasets

# Google Drive에 저장한 json 파일 불러오기
import json

from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/sample_data/travel_chatbot_data(1).json'

from datasets import Dataset

# 데이터셋 로드
with open(file_path, 'r') as file:
    data = json.load(file)

# Hugging Face 데이터셋 변환
# dataset = Dataset.from_dict({
#     "prompt": [item["prompt"] for item in data],
#     "response": [item["response"] for item in data]
# })

from sklearn.model_selection import train_test_split

# 데이터 분할 (80% 학습, 20% 평가)
train_data, eval_data = train_test_split(data, test_size=0.2)
# 학습 및 평가 데이터셋 변환
train_dataset = Dataset.from_dict({
    "prompt": [item["prompt"] for item in train_data],
    "response": [item["response"] for item in train_data]
})
eval_dataset = Dataset.from_dict({
    "prompt": [item["prompt"] for item in eval_data],
    "response": [item["response"] for item in eval_data]
})


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



***label 크기와 input_ids 크기가 달라서 오류 발생함. (입력 데이터, 출력 데이터 크기 차이)

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback


# 학습 데이터셋 준비
# def preprocess_function(examples):
#     inputs = [f"Q: {q}\nA:" for q in examples["prompt"]]
#     targets = examples["response"]
#     model_inputs = tokenizer(inputs, padding=True, truncation=True, max_length=50)
#     labels = tokenizer(targets, padding="max_length", truncation=True, max_length=50)
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs


def preprocess_function(examples):
    inputs = [f"Q: {q}\nA:" for q in examples["prompt"]]
    targets = examples["response"]
    # 입력 텍스트 토큰화
    model_inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=200,
        return_tensors="pt"  # PyTorch 텐서 반환
    )
    # 출력 텍스트 토큰화 (레이블)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            padding="max_length",  # 고정된 길이로 패딩
            truncation=True,
            max_length=200,
            return_tensors="pt"
        )
    labels["input_ids"] = torch.tensor(labels["input_ids"])
    # 패딩 토큰을 -100으로 설정 (loss 계산 제외)
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in example]
        for example in labels["input_ids"]
    ]
    labels["input_ids"] = torch.tensor(labels["input_ids"])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# 데이터 전처리
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)
# processed_dataset = dataset.map(preprocess_function, batched=True)

# 샘플 데이터 확인
# print(train_dataset.shape)
# print(eval_dataset.shape)
# print(train_dataset[:2])
# print(eval_dataset[:2])

# 데이터셋에서 첫 번째 샘플 확인
print(train_dataset[0])
# 입력 데이터의 크기 확인
print(len(train_dataset[0]["input_ids"]))
# 입력 토큰의 길이
print(len(train_dataset[0]["labels"]))
output = model.generate(
    input_ids,
    max_new_tokens=200,  # 새로 생성할 토큰 길이
    num_return_sequences=1,  # 생성할 텍스트 시퀀스 수
    do_sample=false,  # 랜덤 샘플링 활성화
    top_k=50,  # 상위 K개의 토큰만 고려
    top_p=0.95,  # 상위 P 확률의 토큰만 고려
    temperature=0.8,  # 텍스트 다양성 조정
    repetition_penalty=1.2,  # 반복 억제
    early_stopping=True  # 조건 충족 시 조기 종료
)

# 결과 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print("Generated Text:", generated_text)
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",        # 체크포인트 저장도 매 에폭마다 수행
    learning_rate=3e-5,
    per_device_train_batch_size=2,  # 배치사이즈 조정
    num_train_epochs=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    report_to="none",  # wandb 비활성화
    load_best_model_at_end = True,
    metric_for_best_model="eval_loss"  # 성능 기준 메트릭 (기본값: eval_loss)
)
output = model.generate(
    input_ids,
    max_new_tokens=50,  # 새로 생성할 토큰 길이
    num_return_sequences=1,  # 생성할 텍스트 시퀀스 수
    do_sample=True,  # 랜덤 샘플링 활성화
    top_k=50,  # 상위 K개의 토큰만 고려
    top_p=0.95,  # 상위 P 확률의 토큰만 고려
    temperature=0.8,  # 텍스트 다양성 조정
    repetition_penalty=1.2,  # 반복 억제
    early_stopping=True  # 조건 충족 시 조기 종료
)

# 결과 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print("Generated Text:", generated_text)
# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,   # Pass the evaluation dataset
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # 손실이 3번 연속 개선되지 않으면 중단
)

# Fine-Tuning 시작
trainer.train()


Map:   0%|          | 0/585 [00:00<?, ? examples/s]

<ipython-input-24-f9dfcb1e35fa>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels["input_ids"] = torch.tensor(labels["input_ids"])


Map:   0%|          | 0/147 [00:00<?, ? examples/s]

{'prompt': '전라남도 순천시에서 갈만한 여행지 추천해줘.', 'response': '순천생태마을 - 산간마을의 오염되지 않은 환경 속에 도심에서는 볼 수 없는 생명들이 함께하는 곳으로 계곡과 습지가 잘 보존되어 있어 생태교육장으로 안성맞춤', 'input_ids': [16026, 401, 14103, 20392, 22625, 9546, 13342, 12079, 8263, 13815, 8711, 8244, 9585, 407, 401, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Generated Text: 서울에서 갈만한 여행지 추천해줘. 최근 한 온라인 커뮤니티 게시판에는 '포스트잇'이라는 제목으로 사진이 게재됐다.
공개된 사진 속에는 화이트 와인과 맥주잔에 각각 페트와인이 잔뜩 담겨있는 모습이 담겼다.
특히 이 사진을 접한 네티즌들은 "페


<ipython-input-24-f9dfcb1e35fa>:109: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,8.499991
2,8.106200,8.505743
3,8.106200,8.596268
4,7.396500,8.710530


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=1172, training_loss=7.658417477135772, metrics={'train_runtime': 195.1181, 'train_samples_per_second': 29.982, 'train_steps_per_second': 15.017, 'total_flos': 238837248000000.0, 'train_loss': 7.658417477135772, 'epoch': 4.0})

###4. 추론

In [ ]:
# Fine-Tuned 모델 로드
model.save_pretrained('./fine_tuned_kogpt')
tokenizer.save_pretrained('./fine_tuned_kogpt')

# Fine-Tuned 모델 로드 후 추론
fine_tuned_model = GPT2LMHeadModel.from_pretrained('./fine_tuned_kogpt')
fine_tuned_tokenizer = PreTrainedTokenizerFast.from_pretrained('./fine_tuned_kogpt')

# Fine-Tuned 모델 사용
prompt = "서울에서 갈만한 문화 장소 1곳 추천해줘."
input_ids = fine_tuned_tokenizer.encode(prompt, return_tensors='pt')
output = fine_tuned_model.generate(input_ids, max_length=50, do_sample=True, top_k=50, top_p=0.95)
print(fine_tuned_tokenizer.decode(output[0], skip_special_tokens=True))


서울에서 갈만한 문화 장소 추천해줘. 영화 - 문화관 시절 영화센터 박해박물관 박해 일제강점기 박해수문화의 서정 1934년 삶과 어우러져극 조선시대 옛 조선시대 일제강점기 3대 윤봉 - 윤봉 - 윤봉 - 윤봉지 개관기념 윤봉 - 윤봉 - 윤봉 - 윤봉
